# HRM (Hierarchical Reasoning Model) Trading Training on Colab/Kaggle

This notebook converts the HRM training pipeline to run on Google Colab or Kaggle notebooks with automatic TPU/GPU/CPU detection.

## Overview
- **HRM Architecture**: Brain-inspired hierarchical model with 27M parameters
- **Training Approach**: Deep supervision with adaptive computation time
- **Data Processing**: Automated pipeline from raw to training-ready data
- **Hardware Support**: Automatic TPU/GPU/CPU detection with parallel training

Based on the research paper: *Hierarchical Reasoning Model* by Guan Wang et al.

## 1. Setup Environment and Clone Repository

Clone the private GitHub repository using the service token.

In [1]:
import os
import sys
import subprocess

# Detect runtime environment
def is_colab():
    return "COLAB_GPU" in os.environ

def is_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def check_tpu_availability():
    try:
        import requests
        response = requests.get(
            'http://metadata.google.internal/computeMetadata/v1/instance/attributes/accelerator-type',
            headers={'Metadata-Flavor': 'Google'},
            timeout=5
        )
        return 'tpu' in response.text.lower()
    except:
        return False

in_cloud_env = is_colab() or is_kaggle()
is_tpu_environment = check_tpu_availability() if in_cloud_env else False

print(f"🔍 Running in Colab: {is_colab()} | Kaggle: {is_kaggle()} | TPU: {is_tpu_environment}")

# Repo details
repo_url = "https://{personal_access_token}@github.com/marshaltudu14/AlgoTrading.git"
repo_path = "/content/AlgoTrading"

if in_cloud_env or is_tpu_environment:
    # Clone repository only if running in cloud
    if not os.path.exists(repo_path):
        print("📥 Cloning AlgoTrading repository...")
        result = subprocess.run(
            ["git", "clone", repo_url, repo_path],
            capture_output=True, text=True
        )
        if result.returncode == 0:
            print("✅ Repository cloned successfully!")
        else:
            print(f"❌ Failed to clone repository: {result.stderr}")
            raise Exception("Repository clone failed")
    else:
        print("✅ Repository already exists")

    # Change to repository directory
    os.chdir(repo_path)
    sys.path.append(repo_path)

    print(f"📁 Current directory: {os.getcwd()}")
    print(f"📋 Repository contents:")
    for item in sorted(os.listdir('.')):
        print(f"  {item}")
else:
    print("⏭️ Local environment detected. Skipping repository cloning.")

🔍 Running in Colab: True | Kaggle: False | TPU: False
📥 Cloning AlgoTrading repository...
✅ Repository cloned successfully!
📁 Current directory: /content/AlgoTrading
📋 Repository contents:
  .bmad-core
  .claude
  .gemini
  .git
  .gitignore
  .roomodes
  HRM_Training_Notebook.ipynb
  __pycache__
  config
  data
  data_processing_pipeline.log
  docs
  fyers_docs.txt
  hrm-research-paper.txt
  qwen-cli-claude-delegation.md
  requirements.txt
  run_training.py
  src
  tests
  web-bundles


## 2. Install Dependencies

Install all required packages including TPU support for Google Colab.

In [2]:
import os
import sys

# Detect runtime environment
def is_colab():
    return "COLAB_GPU" in os.environ

def is_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

def check_tpu_availability():
    try:
        import requests
        response = requests.get(
            'http://metadata.google.internal/computeMetadata/v1/instance/attributes/accelerator-type',
            headers={'Metadata-Flavor': 'Google'},
            timeout=5
        )
        return 'tpu' in response.text.lower()
    except:
        return False

in_cloud_env = is_colab() or is_kaggle()
is_tpu_environment = check_tpu_availability() if in_cloud_env else False

print(f"🔍 Running in Colab: {is_colab()} | Kaggle: {is_kaggle()} | TPU: {is_tpu_environment}")

# Install only if in cloud (Colab/Kaggle/TPU)
if in_cloud_env or is_tpu_environment:
    print("📦 Installing base requirements...")
    !pip install -r requirements.txt

    if is_tpu_environment:
        print("🚀 Installing TPU support...")
        !pip install torch_xla cloud-tpu-client

    print("📊 Installing visualization libraries...")
    !pip install ipywidgets plotly kaleido

    print("✅ All dependencies installed successfully!")
else:
    print("⏭️ Local environment detected. Skipping installation.")

🔍 Running in Colab: True | Kaggle: False | TPU: False
📦 Installing base requirements...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jupyter-server to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB

📊 Installing visualization libraries...
✅ All dependencies installed successfully!


In [1]:
import numpy as np
np.NaN = np.nan  # Fix missing alias
import pandas_ta as ta

In [2]:
import os
import sys
from pathlib import Path

# Detect environment and set project path
def is_colab():
    return "COLAB_GPU" in os.environ

def is_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

# Determine project path based on environment
in_cloud_env = is_colab() or is_kaggle()

if in_cloud_env:
    # Cloud environment: Use cloned repository path
    PROJECT_PATH = "/content/AlgoTrading"
    print(f"☁️ Cloud environment detected")
else:
    # Local environment: Use current directory or set custom path
    # You can modify this path if your project is in a different location
    PROJECT_PATH = os.getcwd()
    print(f"💻 Local environment detected")

# Convert to Path object for easier manipulation
project_path = Path(PROJECT_PATH)

print(f"📁 Project path set to: {project_path}")
print(f"📂 Project exists: {project_path.exists()}")

# Change to project directory if it exists
if project_path.exists():
    os.chdir(str(project_path))
    print(f"✅ Changed working directory to: {os.getcwd()}")

    # Add project path to Python path for imports
    if str(project_path) not in sys.path:
        sys.path.insert(0, str(project_path))
        print(f"✅ Added to Python path: {project_path}")

    # Verify key directories exist
    key_dirs = ['src', 'config', 'data']
    for dir_name in key_dirs:
        dir_path = project_path / dir_name
        if dir_path.exists():
            print(f"✅ Found directory: {dir_name}/")
        else:
            print(f"⚠️ Directory not found: {dir_name}/")

    # List contents to verify
    print(f"\n📋 Project contents:")
    for item in sorted(project_path.iterdir()):
        if item.is_dir():
            print(f"  📁 {item.name}/")
        else:
            print(f"  📄 {item.name}")
else:
    print(f"❌ Project path does not exist: {project_path}")
    print("⚠️ You may need to modify PROJECT_PATH variable above")

# Store project path for use in other cells
globals()['PROJECT_PATH'] = str(project_path)
print(f"\n🎯 PROJECT_PATH variable set: {PROJECT_PATH}")

☁️ Cloud environment detected
📁 Project path set to: /content/AlgoTrading
📂 Project exists: True
✅ Changed working directory to: /content/AlgoTrading
✅ Added to Python path: /content/AlgoTrading
✅ Found directory: src/
✅ Found directory: config/
✅ Found directory: data/

📋 Project contents:
  📁 .bmad-core/
  📁 .claude/
  📁 .gemini/
  📁 .git/
  📄 .gitignore
  📄 .roomodes
  📄 HRM_Training_Notebook.ipynb
  📁 __pycache__/
  📁 config/
  📁 data/
  📄 data_processing_pipeline.log
  📁 docs/
  📄 fyers_docs.txt
  📄 hrm-research-paper.txt
  📁 logs/
  📄 qwen-cli-claude-delegation.md
  📁 reports/
  📄 requirements.txt
  📄 run_training.py
  📁 src/
  📁 temp/
  📁 tests/
  📁 web-bundles/

🎯 PROJECT_PATH variable set: /content/AlgoTrading


## 3. Device Detection and Setup

Automatically detect and configure the best available device (TPU > GPU > CPU).

In [3]:
import torch
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Import optimized components
from src.utils.device_manager import get_device_manager
from src.utils.training_optimizer import get_training_optimizer

# Initialize device manager
device_manager = get_device_manager()
device = device_manager.get_device()
device_type = device_manager.get_device_type()

# Print detailed device information
device_manager.print_device_summary()

# Initialize high-performance training optimizer
training_optimizer = get_training_optimizer()
performance_config = training_optimizer.get_optimized_training_config()

# Print performance optimization summary
training_optimizer.print_performance_summary()

# Store optimized configuration for training
DEVICE = device
DEVICE_TYPE = device_type
BATCH_SIZE = performance_config['batch_size']  # Optimized for 15GB VRAM
GRADIENT_ACCUMULATION = performance_config['gradient_accumulation_steps']
EFFECTIVE_BATCH_SIZE = performance_config['effective_batch_size']
MIXED_PRECISION = performance_config['mixed_precision']
DATALOADER_CONFIG = performance_config['dataloader_config']

print(f"\n🎯 HIGH-PERFORMANCE TRAINING CONFIGURATION:")
print(f"  Device: {DEVICE}")
print(f"  Batch Size: {BATCH_SIZE} (optimized for 15GB VRAM)")
print(f"  Gradient Accumulation: {GRADIENT_ACCUMULATION}x")
print(f"  Effective Batch Size: {EFFECTIVE_BATCH_SIZE}")
print(f"  Mixed Precision: {'✅ Enabled' if MIXED_PRECISION else '❌ Disabled'}")
print(f"  DataLoader Workers: {DATALOADER_CONFIG['num_workers']}")
print(f"  Expected Speedup: 5-10x faster than default settings")


🔥 HIGH-PERFORMANCE DEVICE CONFIGURATION
Selected Device: cuda
Device Type: GPU
GPU Name: Tesla T4
GPU Memory: 14.7 GB total
Available Memory: 14.7 GB
GPU Count: 1
Compute Capability: 7
Tensor Cores: ✅ Available
Mixed Precision: ✅ Enabled

🚀 PERFORMANCE OPTIMIZATIONS:
Recommended Batch Size: 256
Gradient Accumulation: 4x
DataLoader Workers: 7
Pin Memory: ✅ Enabled
Mixed Precision: ✅ FP16 Enabled (Tensor Cores)


🚀 HIGH-PERFORMANCE TRAINING CONFIGURATION
Target Hardware: 15GB VRAM + 12GB RAM (Colab/Kaggle)
Device: cuda
Batch Size: 256 (8x larger for 15GB VRAM)
Gradient Accumulation: 4x
Effective Batch Size: 1024 (massive training batches)
Mixed Precision: ✅ FP16 Enabled
DataLoader Workers: 7
Pin Memory: ✅ Enabled
Model Compilation: ✅ Enabled

⚡ EXPECTED PERFORMANCE IMPROVEMENTS:
• 8x larger batch sizes = Better gradient estimates
• Mixed precision = 1.5-2x speed boost with Tensor Cores
• Optimized data loading = 2-3x faster data pipeline
• Memory management = Stable training with full V

/content/AlgoTrading/src/utils/training_optimizer.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler() if self.use_mixed_precision else None


## 4. Data Processing Pipeline

Run the data processing pipeline to convert raw data into training-ready features.

In [4]:
import sys
import os
from pathlib import Path

# Import data processing pipeline
from src.data_processing.pipeline import DataProcessingPipeline

print("🔄 Initializing Data Processing Pipeline...")
print("=" * 80)

# Initialize pipeline
pipeline = DataProcessingPipeline()

# Check if raw data exists
raw_data_path = Path('data/raw')
final_data_path = Path('data/final')

print(f"📂 Raw data path: {raw_data_path}")
print(f"📂 Final data path: {final_data_path}")

# List raw data files
if raw_data_path.exists():
    raw_files = list(raw_data_path.glob("*.csv"))
    print(f"📄 Found {len(raw_files)} raw data files:")
    for file in raw_files[:5]:  # Show first 5 files
        print(f"  - {file.name}")
    if len(raw_files) > 5:
        print(f"  ... and {len(raw_files) - 5} more files")
else:
    print("⚠️ No raw data directory found")

# Run feature generation pipeline
print("\n🚀 Running Feature Generation Pipeline...")
print("=" * 80)

try:
    result = pipeline.run_complete_pipeline(
        input_dir=str(raw_data_path),
        output_dir=str(final_data_path)
    )

    if result['success']:
        print("\n✅ Data Processing Pipeline Completed Successfully!")
        print("=" * 80)
        print(f"📊 Summary:")
        print(f"  • Total files processed: {result.get('total_files_processed', 'Unknown')}")
        print(f"  • Total rows processed: {result.get('total_rows_processed', 0):,}")
        print(f"  • Processing time: {result.get('total_time_formatted', 'Unknown')}")
        print(f"  • Output directory: {result.get('output_directory', 'Unknown')}")

        # List final data files
        if final_data_path.exists():
            final_files = list(final_data_path.glob("features_*.csv"))
            print(f"\n📈 Generated {len(final_files)} feature files:")
            for file in final_files:
                print(f"  - {file.name}")
    else:
        print(f"❌ Data processing failed: {result.get('error', 'Unknown error')}")
        raise Exception("Data processing pipeline failed")

except Exception as e:
    print(f"❌ Pipeline execution failed: {str(e)}")
    # Continue anyway if some data exists
    if final_data_path.exists() and list(final_data_path.glob("features_*.csv")):
        print("⚠️ Using existing processed data files")
    else:
        raise

print("\n🎯 Data processing complete. Ready for training!")

🔄 Initializing Data Processing Pipeline...
📂 Raw data path: data/raw
📂 Final data path: data/final
📄 Found 60 raw data files:
  - Sensex_5.csv
  - Bankex_45.csv
  - Finnifty_120.csv
  - Bankex_15.csv
  - Bankex_240.csv
  ... and 55 more files

🚀 Running Feature Generation Pipeline...

✅ Data Processing Pipeline Completed Successfully!
📊 Summary:
  • Total files processed: 60
  • Total rows processed: 664,905
  • Processing time: 9.3 minutes
  • Output directory: data/final

📈 Generated 60 feature files:
  - features_Sensex_2.csv
  - features_Bankex_180.csv
  - features_Finnifty_2.csv
  - features_Finnifty_45.csv
  - features_Bankex_2.csv
  - features_Nifty_180.csv
  - features_Bank_Nifty_3.csv
  - features_Bankex_5.csv
  - features_Finnifty_3.csv
  - features_Sensex_240.csv
  - features_Bank_Nifty_60.csv
  - features_Finnifty_120.csv
  - features_Bankex_20.csv
  - features_Nifty_20.csv
  - features_Nifty_2.csv
  - features_Sensex_15.csv
  - features_Nifty_3.csv
  - features_Nifty_10.cs

## 5. HRM Training Configuration

Configure the HRM training parameters and initialize the trainer.

In [5]:
# HIGH-PERFORMANCE Training parameters for Colab/Kaggle (15GB VRAM + 12GB RAM)
TRAINING_PARAMS = {
    'epochs': 100,  # More epochs due to faster training
    'save_frequency': 20,  # Save checkpoint every 20 epochs
    'log_frequency': 5,   # Log progress every 5 epochs
    'validation_frequency': 10,  # Validate every 10 epochs
    'debug_mode': True,   # Disable debug mode for speed
    'config_path': 'config/hrm_config.yaml',
    'data_path': 'data/final',
    'memory_efficient': False,  # Disabled - we have plenty of VRAM now

    # High-performance optimizations
    'high_performance': True,
    'mixed_precision': MIXED_PRECISION,
    'gradient_accumulation_steps': GRADIENT_ACCUMULATION,
    'dataloader_config': DATALOADER_CONFIG,
    'compile_model': True,  # PyTorch 2.0+ optimization

    # Optimized for cloud environments
    'cloud_optimized': True,
    'vram_target': '15GB',
    'ram_target': '12GB'
}

print(f"\n🚀 HIGH-PERFORMANCE Training Parameters (Colab/Kaggle Optimized):")
for key, value in TRAINING_PARAMS.items():
    if isinstance(value, dict):
        print(f"  • {key}:")
        for subkey, subvalue in value.items():
            print(f"    - {subkey}: {subvalue}")
    else:
        print(f"  • {key}: {value}")

print(f"\n📱 Device Configuration:")
print(f"  • Device: {DEVICE}")
print(f"  • Device Type: {DEVICE_TYPE}")
print(f"  • Batch Size: {BATCH_SIZE} (8x larger for 15GB VRAM)")
print(f"  • Effective Batch Size: {EFFECTIVE_BATCH_SIZE} (with gradient accumulation)")

print(f"\n🏗️ HIGH-PERFORMANCE Training Architecture:")
print(f"  • Multi-Data per Epoch: All instruments trained in each epoch")
print(f"  • Memory Strategy: Full VRAM utilization (15GB)")
print(f"  • Mixed Precision: {'FP16 Enabled' if MIXED_PRECISION else 'FP32'}")
print(f"  • DataLoader Workers: {DATALOADER_CONFIG['num_workers']} (optimized for 12GB RAM)")
print(f"  • Expected Performance: 5-10x faster than default settings")
print(f"  • Flow: Large Batches → Mixed Precision → Gradient Accumulation → Maximum Speed")


🚀 HIGH-PERFORMANCE Training Parameters (Colab/Kaggle Optimized):
  • epochs: 100
  • save_frequency: 20
  • log_frequency: 5
  • validation_frequency: 10
  • debug_mode: True
  • config_path: config/hrm_config.yaml
  • data_path: data/final
  • memory_efficient: False
  • high_performance: True
  • mixed_precision: True
  • gradient_accumulation_steps: 4
  • dataloader_config:
    - num_workers: 7
    - pin_memory: True
    - prefetch_factor: 2
    - persistent_workers: True
    - batch_size: 256
  • compile_model: True
  • cloud_optimized: True
  • vram_target: 15GB
  • ram_target: 12GB

📱 Device Configuration:
  • Device: cuda
  • Device Type: gpu
  • Batch Size: 256 (8x larger for 15GB VRAM)
  • Effective Batch Size: 1024 (with gradient accumulation)

🏗️ HIGH-PERFORMANCE Training Architecture:
  • Multi-Data per Epoch: All instruments trained in each epoch
  • Memory Strategy: Full VRAM utilization (15GB)
  • Mixed Precision: FP16 Enabled
  • DataLoader Workers: 7 (optimized for 12

## 6. Initialize HRM Training Pipeline

Initialize the HRM training pipeline with automatic device detection and parallel training support.

In [6]:
# Import the HRM trainer directly instead of using run_training pipeline
from src.models.hrm_trainer import HRMTrainer
import logging

# Setup enhanced logging for notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Only console output for notebook
    ]
)
logger = logging.getLogger(__name__)

print("🚀 Initializing HRM Training Pipeline (Multi-Data per Epoch)...")
print("=" * 80)

try:
    # Initialize HRM trainer with memory-efficient multi-data training
    trainer = HRMTrainer(
        config_path=TRAINING_PARAMS['config_path'],
        data_path=TRAINING_PARAMS['data_path'],
        device=str(DEVICE),
        debug_mode=TRAINING_PARAMS['debug_mode']  # Now False for multi-data
    )

    print("✅ HRM Trainer initialized successfully!")

    # Check available instruments for multi-data training
    from pathlib import Path
    data_files = list(Path(TRAINING_PARAMS['data_path']).glob("features_*.csv"))
    available_instruments = [f.stem.replace('features_', '') for f in data_files]

    print(f"📊 Available instruments: {len(available_instruments)}")
    print("💾 Memory-efficient training: Loading one data file at a time")

    # List available instruments
    if available_instruments:
        print("\n📈 Available Training Instruments (All will be used per epoch):")
        for i, instrument in enumerate(available_instruments, 1):
            print(f"  {i}. {instrument}")

        print(f"\n🎯 Training Architecture:")
        print(f"  • Per Epoch: All {len(available_instruments)} instruments")
        print(f"  • Per Instrument: Multiple episodes (1500 rows each)")
        print(f"  • Memory Usage: One data file loaded at a time")
        print(f"  • Training Flow: Instrument 1 → All episodes → Instrument 2 → All episodes → ... → Epoch Complete")
    else:
        raise ValueError("No instruments available for training")

except Exception as e:
    print(f"❌ Failed to initialize HRM trainer: {str(e)}")
    import traceback
    traceback.print_exc()
    raise

print("\n🔥 Ready to start multi-data HRM training!")

🚀 Initializing HRM Training Pipeline (Multi-Data per Epoch)...
✅ HRM Trainer initialized successfully!
📊 Available instruments: 60
💾 Memory-efficient training: Loading one data file at a time

📈 Available Training Instruments (All will be used per epoch):
  1. Sensex_2
  2. Bankex_180
  3. Finnifty_2
  4. Finnifty_45
  5. Bankex_2
  6. Nifty_180
  7. Bank_Nifty_3
  8. Bankex_5
  9. Finnifty_3
  10. Sensex_240
  11. Bank_Nifty_60
  12. Finnifty_120
  13. Bankex_20
  14. Nifty_20
  15. Nifty_2
  16. Sensex_15
  17. Nifty_3
  18. Nifty_10
  19. Sensex_180
  20. Sensex_120
  21. Bankex_45
  22. Bank_Nifty_180
  23. Finnifty_240
  24. Nifty_240
  25. Sensex_60
  26. Bank_Nifty_15
  27. Bank_Nifty_2
  28. Bank_Nifty_120
  29. Bank_Nifty_30
  30. Finnifty_60
  31. Sensex_5
  32. Bank_Nifty_20
  33. Finnifty_10
  34. Finnifty_30
  35. Bank_Nifty_45
  36. Sensex_30
  37. Bankex_240
  38. Bankex_120
  39. Finnifty_20
  40. Bankex_10
  41. Bankex_15
  42. Nifty_60
  43. Finnifty_5
  44. Sensex_3


## 7. Training Loss Visualization Setup

Setup real-time visualization for training metrics and loss curves.

In [7]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from IPython.display import display, clear_output
import threading
import time

class HRMTrainingVisualizer:
    """Real-time training visualization for Jupyter notebooks"""

    def __init__(self):
        self.training_history = []
        self.loss_history = []
        self.reward_history = []
        self.best_reward = float('-inf')

        # Setup plots
        self.fig = None
        self.setup_plots()

    def setup_plots(self):
        """Initialize the plotting framework"""
        plt.style.use('default')
        plt.rcParams['figure.figsize'] = (15, 10)

    def update_metrics(self, episode, metrics):
        """Update training metrics"""
        self.training_history.append({
            'episode': episode,
            'total_reward': metrics.get('total_reward', 0),
            'avg_reward': metrics.get('avg_reward', 0),
            'total_loss': metrics.get('total_loss', 0),
            'avg_loss': metrics.get('avg_loss', 0),
            'steps': metrics.get('steps', 0)
        })

        self.reward_history.append(metrics.get('avg_reward', 0))
        self.loss_history.append(metrics.get('avg_loss', 0))

        if metrics.get('avg_reward', 0) > self.best_reward:
            self.best_reward = metrics.get('avg_reward', 0)

    def create_training_plots(self):
        """Create comprehensive training plots"""
        if len(self.training_history) < 2:
            return

        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Average Reward per Episode', 'Average Loss per Episode',
                          'Total Reward Trend', 'Training Progress Summary'),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"secondary_y": False}]]
        )

        episodes = [h['episode'] for h in self.training_history]

        # Reward plot
        fig.add_trace(
            go.Scatter(x=episodes, y=self.reward_history,
                      mode='lines+markers', name='Avg Reward',
                      line=dict(color='green', width=2)),
            row=1, col=1
        )

        # Loss plot
        fig.add_trace(
            go.Scatter(x=episodes, y=self.loss_history,
                      mode='lines+markers', name='Avg Loss',
                      line=dict(color='red', width=2)),
            row=1, col=2
        )

        # Total reward trend
        total_rewards = [h['total_reward'] for h in self.training_history]
        fig.add_trace(
            go.Scatter(x=episodes, y=total_rewards,
                      mode='lines+markers', name='Total Reward',
                      line=dict(color='blue', width=2)),
            row=2, col=1
        )

        # Summary metrics
        if len(self.reward_history) >= 10:
            recent_avg_reward = np.mean(self.reward_history[-10:])
            recent_avg_loss = np.mean(self.loss_history[-10:])
        else:
            recent_avg_reward = np.mean(self.reward_history)
            recent_avg_loss = np.mean(self.loss_history)

        # Create summary text
        summary_text = f"""
        📊 Training Summary (Last 10 Episodes)
        • Best Reward: {self.best_reward:.4f}
        • Recent Avg Reward: {recent_avg_reward:.4f}
        • Recent Avg Loss: {recent_avg_loss:.4f}
        • Total Episodes: {len(self.training_history)}
        """

        fig.add_annotation(
            text=summary_text,
            xref="paper", yref="paper",
            x=0.75, y=0.3, xanchor='left', yanchor='top',
            showarrow=False,
            font=dict(size=12, family="monospace"),
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="black",
            borderwidth=1
        )

        fig.update_layout(
            height=800,
            title_text="HRM Training Progress - Real-time Monitoring",
            showlegend=True
        )

        return fig

    def display_current_metrics(self):
        """Display current training metrics"""
        if not self.training_history:
            return

        latest = self.training_history[-1]

        print(f"\n📈 Episode {latest['episode']} Results:")
        print(f"  🎯 Average Reward: {latest['avg_reward']:.4f}")
        print(f"  📉 Average Loss: {latest['avg_loss']:.4f}")
        print(f"  🎮 Steps Completed: {latest['steps']}")
        print(f"  🏆 Best Reward So Far: {self.best_reward:.4f}")

        if latest['avg_reward'] == self.best_reward:
            print("  🌟 NEW BEST PERFORMANCE! 🌟")

# Initialize visualizer
visualizer = HRMTrainingVisualizer()
print("📊 Training visualizer initialized!")
print("📈 Real-time plots will be updated during training")

📊 Training visualizer initialized!
📈 Real-time plots will be updated during training


## 8. Start HRM Training

Begin the HRM training process with real-time monitoring and visualization.

In [ ]:
import time
from IPython.display import display, clear_output
import ipywidgets as widgets

print("🚀 Starting HRM Multi-Data Training Process...")
print("=" * 80)

# Create training progress widgets for multi-data training
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,  # Will be updated dynamically
    description='Training:',
    bar_style='info',
    style={'bar_color': 'green'},
    orientation='horizontal'
)

status_text = widgets.HTML(value="<b>Initializing multi-data training...</b>")
metrics_text = widgets.HTML(value="")

display(widgets.VBox([progress_bar, status_text, metrics_text]))

# Custom training loop with visualization for multi-data
class HRMTrainingMonitor:
    def __init__(self, trainer, visualizer):
        self.trainer = trainer
        self.visualizer = visualizer
        self.start_time = time.time()

    def run_training_with_monitoring(self, epochs, available_instruments):
        """Run multi-data training with real-time monitoring"""

        # Setup training with complete initialization for multi-data
        print(f"🔧 Setting up multi-data training for {len(available_instruments)} instruments...")
        print(f"📋 Instruments: {available_instruments}")

        status_text.value = f"<b>🎯 Multi-Data Training: {len(available_instruments)} instruments × {epochs} epochs</b>"

        # Calculate total steps for progress bar (memory efficient)
        total_steps = 0
        print("📊 Calculating training steps (memory efficient)...")
        for symbol in available_instruments:
            # Quick data length check (memory efficient)
            from src.utils.data_loader import DataLoader
            temp_loader = DataLoader(final_data_dir=TRAINING_PARAMS['data_path'])
            temp_data = temp_loader.load_final_data_for_symbol(symbol)
            episodes_per_file = max(1, (len(temp_data) - 1) // 1500)  # 1500 = episode_length
            total_steps += episodes_per_file
            del temp_data  # Free memory immediately

        total_steps *= epochs
        progress_bar.max = total_steps

        print(f"📈 Training Progress: {epochs} epochs × {total_steps // epochs} total episodes = {total_steps} steps")
        print("💾 Memory-efficient: One data file loaded at a time")

        # Use the trainer's built-in multi-data training with progress monitoring
        class ProgressCallback:
            def __init__(self, progress_bar, status_text, metrics_text, visualizer):
                self.progress_bar = progress_bar
                self.status_text = status_text
                self.metrics_text = metrics_text
                self.visualizer = visualizer
                self.step_count = 0

            def update(self, epoch, instrument_idx, total_instruments, episode_metrics=None):
                self.step_count += 1
                self.progress_bar.value = self.step_count

                current_instrument = available_instruments[instrument_idx] if instrument_idx < len(available_instruments) else "Completing"

                self.status_text.value = f"""
                <b>📈 Epoch {epoch + 1}/{epochs} - Processing: {current_instrument}</b><br>
                📊 Instrument {instrument_idx + 1}/{total_instruments}<br>
                💾 Memory: One data file loaded at a time
                """

                if episode_metrics:
                    self.visualizer.update_metrics(self.step_count, episode_metrics)

                    self.metrics_text.value = f"""
                    <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
                    <b>📊 Current Metrics:</b><br>
                    🎯 Avg Reward: <span style="color: green;"><b>{episode_metrics.get('avg_reward', 0):.4f}</b></span><br>
                    📉 Avg Loss: <span style="color: red;"><b>{episode_metrics.get('avg_loss', 0):.4f}</b></span><br>
                    🎮 Steps: {episode_metrics.get('steps', 0)}<br>
                    🏆 Best Reward: <span style="color: blue;"><b>{self.visualizer.best_reward:.4f}</b></span>
                    </div>
                    """

        # Custom progress callback (simplified for now)
        try:
            print("🚀 Starting multi-data training...")
            training_history = self.trainer.train(
                epochs=epochs,
                available_instruments=available_instruments,
                save_frequency=TRAINING_PARAMS['save_frequency'],
                log_frequency=TRAINING_PARAMS['log_frequency']
            )

            # Update visualizer with final results
            for i, metrics in enumerate(training_history):
                self.visualizer.update_metrics(i + 1, metrics)

            # Final status update
            status_text.value = "<b>✅ Multi-data training completed successfully!</b>"
            progress_bar.bar_style = 'success'

            return training_history

        except KeyboardInterrupt:
            print("\n⚠️ Training interrupted by user")
            status_text.value = "<b>⚠️ Training interrupted by user</b>"
            progress_bar.bar_style = 'warning'
            return self.trainer.training_history
        except Exception as e:
            print(f"❌ Training error: {str(e)}")
            status_text.value = f"<b>❌ Training error: {str(e)}</b>"
            progress_bar.bar_style = 'danger'
            import traceback
            traceback.print_exc()
            return self.trainer.training_history

# Initialize training monitor
monitor = HRMTrainingMonitor(trainer, visualizer)

# Start multi-data training
try:
    training_results = monitor.run_training_with_monitoring(
        epochs=TRAINING_PARAMS['epochs'],
        available_instruments=available_instruments  # Train on ALL instruments per epoch
    )

    print("\n✅ Multi-data training completed successfully!")
    print(f"🎯 Trained on {len(available_instruments)} instruments per epoch")
    print("💾 Memory-efficient: One data file loaded at a time")

except Exception as e:
    print(f"❌ Training failed: {str(e)}")
    import traceback
    traceback.print_exc()

🚀 Starting HRM Multi-Data Training Process...


🔧 Setting up multi-data training for 60 instruments...
📋 Instruments: ['Sensex_2', 'Bankex_180', 'Finnifty_2', 'Finnifty_45', 'Bankex_2', 'Nifty_180', 'Bank_Nifty_3', 'Bankex_5', 'Finnifty_3', 'Sensex_240', 'Bank_Nifty_60', 'Finnifty_120', 'Bankex_20', 'Nifty_20', 'Nifty_2', 'Sensex_15', 'Nifty_3', 'Nifty_10', 'Sensex_180', 'Sensex_120', 'Bankex_45', 'Bank_Nifty_180', 'Finnifty_240', 'Nifty_240', 'Sensex_60', 'Bank_Nifty_15', 'Bank_Nifty_2', 'Bank_Nifty_120', 'Bank_Nifty_30', 'Finnifty_60', 'Sensex_5', 'Bank_Nifty_20', 'Finnifty_10', 'Finnifty_30', 'Bank_Nifty_45', 'Sensex_30', 'Bankex_240', 'Bankex_120', 'Finnifty_20', 'Bankex_10', 'Bankex_15', 'Nifty_60', 'Finnifty_5', 'Sensex_3', 'Nifty_15', 'Bankex_3', 'Bank_Nifty_10', 'Nifty_45', 'Bank_Nifty_240', 'Sensex_45', 'Nifty_30', 'Sensex_10', 'Finnifty_15', 'Bankex_60', 'Bank_Nifty_5', 'Nifty_5', 'Nifty_120', 'Bankex_30', 'Sensex_20', 'Finnifty_180']
📊 Calculating training steps (memory efficient)...
📈 Training Progress: 100 epochs × 440 

## 9. Training Results Analysis

Analyze the training results and evaluate model performance.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

print("📊 Analyzing Training Results...")
print("=" * 80)

# Final training visualization
if visualizer.training_history:
    print("📈 Generating Final Training Report...")

    # Create comprehensive plots
    final_fig = visualizer.create_training_plots()
    if final_fig:
        final_fig.update_layout(title_text="HRM Training Results - Final Report")
        final_fig.show()

    # Statistical analysis
    rewards = visualizer.reward_history
    losses = visualizer.loss_history

    print(f"\n📈 Training Statistics:")
    print(f"  🎯 Total Episodes Completed: {len(rewards)}")
    print(f"  🏆 Best Reward Achieved: {visualizer.best_reward:.4f}")
    print(f"  📊 Final Average Reward: {rewards[-1]:.4f}")
    print(f"  📉 Final Average Loss: {losses[-1]:.4f}")

    if len(rewards) >= 20:
        print(f"\n📊 Performance Trends:")
        early_reward = np.mean(rewards[:10])
        late_reward = np.mean(rewards[-10:])
        reward_improvement = late_reward - early_reward

        early_loss = np.mean(losses[:10])
        late_loss = np.mean(losses[-10:])
        loss_improvement = early_loss - late_loss

        print(f"  📈 Reward Improvement: {reward_improvement:.4f} ({reward_improvement/early_reward*100:+.1f}%)")
        print(f"  📉 Loss Improvement: {loss_improvement:.4f} ({loss_improvement/early_loss*100:+.1f}%)")

    # Model performance analysis
    print(f"\n🧠 Model Performance Analysis:")
    if len(rewards) > 0:
        reward_variance = np.var(rewards)
        reward_stability = 1.0 / (1.0 + reward_variance) if reward_variance > 0 else 1.0
        print(f"  📊 Reward Variance: {reward_variance:.4f}")
        print(f"  🎯 Training Stability: {reward_stability:.3f}")

        # Performance rating
        if visualizer.best_reward > 0.1:
            performance_rating = "🌟 Excellent"
        elif visualizer.best_reward > 0.05:
            performance_rating = "✅ Good"
        elif visualizer.best_reward > 0.0:
            performance_rating = "⚠️ Fair"
        else:
            performance_rating = "❌ Poor"

        print(f"  🏆 Overall Performance: {performance_rating}")

    # Save training summary
    summary_data = {
        'episode': list(range(1, len(rewards) + 1)),
        'avg_reward': rewards,
        'avg_loss': losses,
        'best_reward_so_far': [max(rewards[:i+1]) for i in range(len(rewards))]
    }

    df_summary = pd.DataFrame(summary_data)

    # Save to CSV
    results_dir = Path("training_results")
    results_dir.mkdir(exist_ok=True)

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    csv_path = results_dir / f"hrm_training_summary_{timestamp}.csv"
    df_summary.to_csv(csv_path, index=False)

    print(f"\n💾 Training summary saved to: {csv_path}")

    # Display final data sample
    print(f"\n📋 Training Summary (Last 10 Episodes):")
    print(df_summary.tail(10).to_string(index=False))

else:
    print("⚠️ No training history available")

print("\n🎉 Training analysis complete!")

## 10. Model Evaluation and Testing

Evaluate the trained HRM model on test data.

In [ ]:
print("🔬 Evaluating Trained HRM Model...")
print("=" * 80)

try:
    # Run model evaluation
    if hasattr(trainer, 'model') and trainer.model is not None:
        print("🧪 Running model evaluation on test episodes...")
        # Use first available instrument for evaluation
        eval_instrument = available_instruments[0] if available_instruments else "Bank_Nifty_5"
        print(f"📊 Evaluating on instrument: {eval_instrument}")

        eval_results = trainer.evaluate(
            episodes=20,  # Evaluate on 20 test episodes
            symbol=eval_instrument
        )

        print("\n📊 Model Evaluation Results:")
        print("=" * 50)

        for metric, value in eval_results.items():
            if isinstance(value, float):
                print(f"  📈 {metric.replace('_', ' ').title()}: {value:.4f}")
            else:
                print(f"  📈 {metric.replace('_', ' ').title()}: {value}")

        # Model summary
        print("\n🧠 HRM Model Architecture Summary:")
        print("=" * 50)

        model_summary = trainer.model.get_model_summary()

        print(f"  🔢 Total Parameters: {model_summary['total_parameters']:,}")
        print(f"  🎓 Trainable Parameters: {model_summary['trainable_parameters']:,}")
        print(f"  🧠 H-Module Parameters: {model_summary['h_module_parameters']:,}")
        print(f"  ⚡ L-Module Parameters: {model_summary['l_module_parameters']:,}")
        print(f"  🔄 ACT Module Parameters: {model_summary['act_module_parameters']:,}")
        print(f"  📚 Deep Supervision Parameters: {model_summary['deep_supervision_parameters']:,}")
        print(f"  📊 Hidden Dimension: {model_summary['hidden_dimension']}")
        print(f"  🔄 H-Cycles: {model_summary['H_cycles']} | L-Cycles: {model_summary['L_cycles']}")
        print(f"  👁️ H-Lookback: {model_summary['h_lookback_window']} | L-Lookback: {model_summary['l_lookback_window']}")
        print(f"  💻 Device: {model_summary['device']}")

        # Checkpoint validation
        print("\n🔍 Validating Model Checkpoints...")
        model_dir = Path("models/hrm")
        checkpoint_dir = Path("checkpoints/hrm")

        checkpoints_valid = False
        if model_dir.exists() or checkpoint_dir.exists():
            checkpoints_valid = True
            print("✅ Model checkpoints are valid and ready for deployment")

            # List saved models
            if model_dir.exists():
                model_files = list(model_dir.glob("*.pt"))
                print(f"\n📁 Saved Models ({len(model_files)}):")
                for model_file in model_files:
                    print(f"  💾 {model_file.name}")

            if checkpoint_dir.exists():
                checkpoint_files = list(checkpoint_dir.glob("*.pt"))
                print(f"\n📁 Training Checkpoints ({len(checkpoint_files)}):")
                for checkpoint_file in checkpoint_files:
                    print(f"  💾 {checkpoint_file.name}")
        else:
            print("⚠️ No checkpoint directories found yet")

    else:
        print("⚠️ No trained model available for evaluation")

except Exception as e:
    print(f"❌ Evaluation failed: {str(e)}")
    import traceback
    traceback.print_exc()

print("\n🎯 Model evaluation complete!")

## 11. Export Results and Model

Export training results, model files, and create deployment package.

In [ ]:
import shutil
import json
import os
from pathlib import Path
from datetime import datetime

print("📦 Creating Export Package...")
print("=" * 80)

# Create export directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_dir = Path(f"hrm_export_{timestamp}")
export_dir.mkdir(exist_ok=True)

print(f"📁 Export directory: {export_dir}")

try:
    # 1. Copy model files
    model_dir = Path("models/hrm")
    if model_dir.exists():
        export_models_dir = export_dir / "models"
        shutil.copytree(model_dir, export_models_dir)
        print(f"✅ Models exported to: {export_models_dir}")

    # 2. Copy checkpoints
    checkpoint_dir = Path("checkpoints/hrm")
    if checkpoint_dir.exists():
        export_checkpoints_dir = export_dir / "checkpoints"
        shutil.copytree(checkpoint_dir, export_checkpoints_dir)
        print(f"✅ Checkpoints exported to: {export_checkpoints_dir}")

    # 3. Copy training results
    results_dir = Path("training_results")
    if results_dir.exists():
        export_results_dir = export_dir / "training_results"
        shutil.copytree(results_dir, export_results_dir)
        print(f"✅ Training results exported to: {export_results_dir}")

    # 4. Copy configuration
    config_files = ['config/hrm_config.yaml']
    export_config_dir = export_dir / "config"
    export_config_dir.mkdir(exist_ok=True)

    for config_file in config_files:
        if Path(config_file).exists():
            shutil.copy2(config_file, export_config_dir)
            print(f"✅ Config exported: {config_file}")

    # 5. Create training summary report
    summary_report = {
        "training_info": {
            "timestamp": timestamp,
            "device_used": str(DEVICE),
            "device_type": DEVICE_TYPE,
            "batch_size": BATCH_SIZE,
            "epochs_completed": len(visualizer.training_history) if visualizer.training_history else 0,
            "selected_instrument": available_instruments[0] if available_instruments else "Unknown"
        },
        "training_parameters": TRAINING_PARAMS,
        "performance_summary": {
            "best_reward": float(visualizer.best_reward) if visualizer.training_history else 0.0,
            "final_reward": float(visualizer.reward_history[-1]) if visualizer.reward_history else 0.0,
            "final_loss": float(visualizer.loss_history[-1]) if visualizer.loss_history else 0.0,
            "total_episodes": len(visualizer.training_history) if visualizer.training_history else 0
        }
    }

    # Add evaluation results if available
    if 'eval_results' in locals():
        summary_report["evaluation_results"] = {
            k: float(v) if isinstance(v, (int, float)) else v
            for k, v in eval_results.items()
        }

    # Save summary report
    summary_file = export_dir / "training_summary.json"
    with open(summary_file, 'w') as f:
        json.dump(summary_report, f, indent=2)
    print(f"✅ Summary report saved: {summary_file}")

    # 6. Create README for export
    readme_content = f"""
# HRM Trading Model Export Package

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Training Device:** {DEVICE_TYPE.upper()}
**Instrument:** {available_instruments[0] if available_instruments else 'Unknown'}

## Contents

- `models/` - Trained HRM models (best model: hrm_best_model.pt)
- `checkpoints/` - Training checkpoints for resuming
- `training_results/` - Training logs and metrics
- `config/` - Model configuration files
- `training_summary.json` - Complete training summary

## Model Architecture

- **Total Parameters:** {model_summary['total_parameters']:,} (≈27M)
- **Architecture:** Hierarchical Reasoning Model (HRM)
- **H-Module Lookback:** {model_summary['h_lookback_window']} candles
- **L-Module Lookback:** {model_summary['l_lookback_window']} candles
- **Hidden Dimension:** {model_summary['hidden_dimension']}

## Performance

- **Best Reward:** {visualizer.best_reward:.4f}
- **Episodes Completed:** {len(visualizer.training_history) if visualizer.training_history else 0}
- **Training Status:** {'Completed' if len(visualizer.training_history) >= TRAINING_PARAMS['epochs'] else 'Partial'}

## 12. Training Summary and Next Steps

Final summary of the training process and recommendations for next steps.

In [ ]:
print("🎊 HRM TRAINING COMPLETED!")
print("=" * 80)

# Final summary
total_time = time.time() - monitor.start_time if 'monitor' in locals() else 0

print(f"\n📊 FINAL TRAINING SUMMARY:")
print(f"=" * 50)
print(f"🧠 Model: Hierarchical Reasoning Model (HRM)")
print(f"📈 Instrument: {selected_instrument if 'selected_instrument' in locals() else 'Unknown'}")
print(f"💻 Device: {DEVICE_TYPE.upper()} ({DEVICE})")
print(f"⏱️ Total Training Time: {total_time/3600:.1f} hours")
print(f"🎯 Episodes Completed: {len(visualizer.training_history) if visualizer.training_history else 0}/{TRAINING_PARAMS['epochs']}")

if visualizer.training_history:
    print(f"🏆 Best Performance: {visualizer.best_reward:.4f}")
    print(f"📈 Final Performance: {visualizer.reward_history[-1]:.4f}")
    print(f"📉 Final Loss: {visualizer.loss_history[-1]:.4f}")

# Model specifications
if 'model_summary' in locals():
    print(f"\n🧠 MODEL SPECIFICATIONS:")
    print(f"=" * 50)
    print(f"📊 Total Parameters: {model_summary['total_parameters']:,}")
    print(f"🔄 H-Cycles: {model_summary['H_cycles']} | L-Cycles: {model_summary['L_cycles']}")
    print(f"👁️ H-Lookback: {model_summary['h_lookback_window']} | L-Lookback: {model_summary['l_lookback_window']}")
    print(f"🔢 Hidden Dimension: {model_summary['hidden_dimension']}")

# Performance evaluation
if visualizer.training_history:
    performance_level = "🌟 Excellent" if visualizer.best_reward > 0.1 else \
                       "✅ Good" if visualizer.best_reward > 0.05 else \
                       "⚠️ Fair" if visualizer.best_reward > 0.0 else \
                       "❌ Needs Improvement"

    print(f"\n🏆 PERFORMANCE EVALUATION: {performance_level}")

# Next steps recommendations
print(f"\n🚀 RECOMMENDED NEXT STEPS:")
print(f"=" * 50)

if visualizer.best_reward > 0.05:
    print(f"✅ Model shows good performance:")
    print(f"  • Deploy model for live testing")
    print(f"  • Run extended evaluation on more instruments")
    print(f"  • Consider ensemble with other models")
elif visualizer.best_reward > 0.0:
    print(f"⚠️ Model shows moderate performance:")
    print(f"  • Continue training for more epochs")
    print(f"  • Experiment with different hyperparameters")
    print(f"  • Try different instruments or timeframes")
else:
    print(f"❌ Model needs improvement:")
    print(f"  • Check data quality and preprocessing")
    print(f"  • Adjust learning rates or architecture")
    print(f"  • Consider curriculum learning approach")

print(f"\n📁 Export package available at: {export_dir if 'export_dir' in locals() else 'Not created'}")

# Additional recommendations
print(f"\n💡 ADDITIONAL RECOMMENDATIONS:")
print(f"=" * 50)
print(f"📊 • Monitor model performance on different market conditions")
print(f"🔄 • Implement continuous learning for market adaptation")
print(f"⚠️ • Add robust risk management and position sizing")
print(f"📈 • Backtest on historical data before live deployment")
print(f"🏭 • Consider distributed training for larger datasets")

print(f"\n🎯 Training session completed successfully!")
print(f"📚 Refer to the HRM research paper for implementation details")
print(f"🔗 Paper: 'Hierarchical Reasoning Model' by Guan Wang et al.")

print("\n" + "=" * 80)
print("🎉 Thank you for using the HRM Training Notebook!")
print("=" * 80)